In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/MyDrive/signature_verification_dataset/sign_data" -d "/content/"

In [3]:
!cp -r "/content/drive/MyDrive/signature_verification_dataset/my_model.h5" -d "/content/"

In [4]:
import os
import sys

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

In [5]:
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import Adam

from keras.models import Model
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications.vgg16 import VGG16, preprocess_input



In [6]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [7]:
def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

In [8]:
def extractor_model(input_shape):

    vgg_16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg_16.trainable = False
    x = vgg_16.output
    x = Flatten(name="flatten")(x)
    output_layer = Dense(128, activation='relu')(x)
    model = Model(inputs=vgg_16.input, outputs=output_layer)
    
    return model

In [14]:
def create_model(input_shape):
  base_model = extractor_model(input_shape)
  print(base_model.summary())
  input_a = Input(shape=input_shape)
  input_b = Input(shape=input_shape)
  feat_1 = base_model(input_a)
  feat_2 = base_model(input_b)
  distance = Lambda(euclidean_distance)([feat_1, feat_2])
  outputs = Dense(1, activation="sigmoid")(distance)

  model = Model([input_a, input_b], outputs=outputs)
  return model

In [15]:
input_shape = (224, 224, 3)

model = create_model(input_shape)
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [11]:
model.load_weights("/content/my_model.h5")

In [13]:
from skimage.io import imread, imshow
from skimage.transform import resize

train_path = "/content/sign_data/train"
test_path = "/content/sign_data/test"

id = input("Enter Id Number: ")
test_img_path = input ("Enter Test Image Path: ")
test_img = imread(os.path.join(test_path,test_img_path))
test_img=resize(test_img,(224,224,3))
test_img = np.expand_dims(test_img, axis=0)

ref_path = os.path.join(test_path, id)
ref_imgs = []
for file in os.listdir(ref_path):
  ref_imgs.append(os.path.join(ref_path,file))

ref_img = imread(ref_imgs[0])
ref_img=resize(ref_img,(224,224,3))
ref_img = np.expand_dims(ref_img, axis=0)

res = model.predict([np.array(ref_img), np.array(test_img)])
val_r = res[0][0]
print("Similarity Distance:", val_r)
if val_r >= 0.6:
  print("Forged")
elif val_r <= 0.4:
    print("Genuin")


Enter Id Number: 068
Enter Test Image Path: 068_forg/03_0124068.PNG
1/1 [==============================] - 0s 35ms/step
Similarity Distance: 0.9874607
Forged
